In [1]:
import numpy as np
from matplotlib import pyplot as plt
from classifier import SGC, GCN
from tqdm import tqdm

import utils

In [2]:
dataset = 'cora'
adj, x, y = utils.load_npz(dataset)
train_nodes, val_nodes, test_nodes = utils.train_val_test_split_tabular(y.shape[0], stratify=y, random_state=15)
edges, time = utils.load_adv_edges(dataset, 'SGA')

In [3]:
model = SGC(adj, x, y, seed=42)
model.train(train_nodes, val_nodes=val_nodes, verbose=False)
f1_micro, f1_macro = model.test(list(edges.keys()))
print(f'【Test】f1 micro {f1_micro:.2%}, f1_macro {f1_macro:.2%}')

【Test】f1 micro 85.60%, f1_macro 83.99%


In [4]:
model = GCN(adj, x, y, seed=42)
model.train(train_nodes, val_nodes=val_nodes, verbose=False)
f1_micro, f1_macro = model.test(list(edges.keys()))
print(f'【Test】f1 micro {f1_micro:.2%}, f1_macro {f1_macro:.2%}')

【Test】f1 micro 85.10%, f1_macro 83.16%


In [5]:
count = 0
pbar = tqdm(edges.items())
for target, edge in pbar:
    modified_adj = utils.flip_adj(adj, edge)
    model = SGC(modified_adj, x, y, seed=42)
    model.train(train_nodes, val_nodes=val_nodes, verbose=False)
    pred = model.predict(target)
    if pred.argmax() == y[target]:
        count += 1
    pbar.set_description(f"Correctly classified {count}, accuracy {count/len(edges):.2%}")

Correctly classified 8, accuracy 0.80%: 100%|██████████| 1000/1000 [09:09<00:00,  1.82it/s]


In [6]:
count = 0
pbar = tqdm(edges.items())
for target, edge in pbar:
    modified_adj = utils.flip_adj(adj, edge)
    model = GCN(modified_adj, x, y, seed=42)
    model.train(train_nodes, val_nodes=val_nodes, verbose=False)
    pred = model.predict(target)
    if pred.argmax() == y[target]:
        count += 1
    pbar.set_description(f"Correctly classified {count}, accuracy {count/len(edges):.2%}")

Correctly classified 13, accuracy 1.30%: 100%|██████████| 1000/1000 [17:38<00:00,  1.06s/it]
